In [ ]:
import itertools
import requests

url = "https://publicapi.solotodo.com/categories/1/browse/?page_size=100&page={}"
results = []
for i in itertools.count(1):
    print(f"Fetching page {i}...")
    response = requests.get(url.format(i))
    response.raise_for_status()
    data = response.json()
    if not data["results"]:
        print("No more results found.")
        break
    results.extend(data["results"])

In [ ]:
len(results)

In [ ]:
import json

with open("results.json", "w") as f:
    json.dump(results, f, indent=2)
    

In [ ]:
import pandas as pd
import json

# Load your data (assuming it's in a variable called 'results')
# If it's in a file, you might load it like:
# with open('your_file.json', 'r') as f:
#     results = json.load(f)

# Extract all product entries from the nested structure
product_entries = []
for item in results:  # results is your list of objects
    for product_entry in item['product_entries']:
        product_entries.append(product_entry)

# Method 1: Using pd.json_normalize() - This is what you were looking for!
df = pd.json_normalize(product_entries)

In [ ]:
list(df.columns)

In [ ]:
import pandas as pd
import re
from collections import defaultdict

def extract_id_mappings(df):
    """
    Extract mappings from ID columns to their related attribute columns
    """
    # Get all column names
    columns = df.columns.tolist()
    
    # Dictionary to store mappings
    id_mappings = {}
    
    # Find all columns ending with '_id'
    id_columns = [col for col in columns if col.endswith('_id')]
    
    print(f"Found {len(id_columns)} ID columns:")
    for col in id_columns:
        print(f"  - {col}")
    
    # For each ID column, find related columns
    for id_col in id_columns:
        # Extract the base name (remove '_id' suffix)
        base_name = id_col[:-3]  # Remove '_id'
        
        # Find all columns that start with the base name
        related_cols = [col for col in columns if col.startswith(base_name) and col != id_col]
        
        if related_cols:
            print(f"\n--- Processing {id_col} ---")
            print(f"Base name: {base_name}")
            print(f"Related columns: {related_cols}")
            
            # Extract unique mappings
            relevant_cols = [id_col] + related_cols
            
            # Get unique combinations of ID and related attributes
            unique_mappings = df[relevant_cols].drop_duplicates().dropna(subset=[id_col])
            
            if len(unique_mappings) > 0:
                print(f"Found {len(unique_mappings)} unique mappings")
                
                # Create mapping dictionary
                mapping_dict = {}
                for _, row in unique_mappings.iterrows():
                    id_value = row[id_col]
                    if pd.notna(id_value):
                        mapping_dict[id_value] = {}
                        for col in related_cols:
                            if col in row and pd.notna(row[col]):
                                # Clean column name for the mapping key
                                clean_col_name = col.replace(base_name + '_', '')
                                mapping_dict[id_value][clean_col_name] = row[col]
                
                id_mappings[id_col] = {
                    'base_name': base_name,
                    'related_columns': related_cols,
                    'mappings': mapping_dict
                }
                
                # Show a sample of the mappings
                print("Sample mappings:")
                for i, (id_val, attrs) in enumerate(list(mapping_dict.items())[:3]):
                    print(f"  ID {id_val}: {attrs}")
                    if i >= 2:  # Show max 3 samples
                        break
                
                if len(mapping_dict) > 3:
                    print(f"  ... and {len(mapping_dict) - 3} more")
    
    return id_mappings

def create_lookup_tables(id_mappings):
    """
    Create individual lookup tables for each ID mapping
    """
    lookup_tables = {}
    
    for id_col, mapping_info in id_mappings.items():
        base_name = mapping_info['base_name']
        mappings = mapping_info['mappings']
        
        if mappings:
            # Convert to DataFrame
            lookup_df = pd.DataFrame.from_dict(mappings, orient='index')
            lookup_df.index.name = base_name + '_id'
            lookup_df = lookup_df.reset_index()
            
            lookup_tables[base_name] = lookup_df
            
            print(f"\nLookup table for {base_name}:")
            print(f"Shape: {lookup_df.shape}")
            print(f"Columns: {lookup_df.columns.tolist()}")
            print("Sample rows:")
            print(lookup_df.head(3))
    
    return lookup_tables

def analyze_id_relationships(df):
    """
    Analyze and extract all ID-based relationships in the dataframe
    """
    print("="*60)
    print("ANALYZING ID-BASED RELATIONSHIPS")
    print("="*60)
    
    # Extract ID mappings
    id_mappings = extract_id_mappings(df)
    
    print("\n" + "="*60)
    print("CREATING LOOKUP TABLES")
    print("="*60)
    
    # Create lookup tables
    lookup_tables = create_lookup_tables(id_mappings)
    
    print("\n" + "="*60)
    print("SUMMARY")
    print("="*60)
    
    print(f"Total ID columns found: {len(id_mappings)}")
    print(f"Lookup tables created: {len(lookup_tables)}")
    
    print("\nAvailable lookup tables:")
    for name, table in lookup_tables.items():
        print(f"  - {name}: {table.shape[0]} unique values, {table.shape[1]} attributes")
    
    return id_mappings, lookup_tables

# Run the analysis
# Assuming your DataFrame is called 'df'
id_mappings, lookup_tables = analyze_id_relationships(df)



In [ ]:
# Save lookup tables to CSV files
from pathlib import Path

print("\nSaving lookup tables to CSV files...")
# Create lookup directory if it doesn't exist
lookup_dir = Path("lookup")
lookup_dir.mkdir(exist_ok=True)

for name, table in lookup_tables.items():
    filename = lookup_dir / f"{name}.csv"
    table.to_csv(filename, index=False)
    print(f"Saved {filename}")

In [ ]:
id_mappings["product.specs.family_line_id"]

In [ ]:
lookup_tables["product.specs.family_line"][["product.specs.family_line_id", "unicode"]].

In [ ]:
import csv
from pathlib import Path

def csv_to_dict(csv_path: str | Path, key_column: str, value_column: str) -> dict[str, str]:
    """
    Create a dictionary from a CSV file using specified columns.
    
    Args:
        csv_path: Path to the CSV file
        key_column: Name of the column to use as dictionary keys
        value_column: Name of the column to use as dictionary values
    
    Returns:
        Dictionary with key_column values as keys and value_column values as values,
        sorted by keys
    
    Raises:
        FileNotFoundError: If CSV file doesn't exist
        KeyError: If specified columns don't exist in CSV
        ValueError: If CSV is empty or malformed
    """
    result = {}
    
    with open(csv_path, 'r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        
        # Check if columns exist
        if key_column not in reader.fieldnames:
            raise KeyError(f"Column '{key_column}' not found in CSV. Available columns: {reader.fieldnames}")
        if value_column not in reader.fieldnames:
            raise KeyError(f"Column '{value_column}' not found in CSV. Available columns: {reader.fieldnames}")
        
        for row in reader:
            key = row[key_column].strip()
            value = row[value_column].strip()
            
            # Skip empty keys
            if key:
                result[key] = value
    
    # Return sorted dictionary
    return dict(sorted(result.items()))

# Usage examples:
mapping = csv_to_dict('lookup/product.specs.keyboard_layout.csv', 'unicode', 'product.specs.keyboard_layout_id')
mapping
# line_mapping = csv_to_dict('product_lines.csv', 'name', 'product.specs.family_line_id')